In [ ]:
import pandas as pd
import re

In [ ]:
df = pd.read_csv("./descriptions_combined_4000.csv")
eight_df = pd.read_csv("./8000_description.csv")
twenty_df = pd.read_csv("./20000_description.csv")
twentyfour_df = pd.read_csv("./20-24000descriptions.csv")
sixteen_df = pd.read_csv("./16000_description.csv")

In [ ]:
# Magic Code (with variants section)
##########################
# df[['word', 'rest']] = df[0].str.extract(r'^([^,]+),\s*(.*)', expand=True)
# df['description'] = df[0].str.extract(r'"(.*?)"', expand=False)
# df['variants'] = df['rest'].str.replace(r'".*?"', '', regex=True).str.strip()
# ################################
# df = df.drop('rest', axis=1)

# df[['word', 'rest']] = df[0].str.extract(r'^([^,]+),\s*(.*)', expand=True)
# df['description'] = df['rest']
# df = df.drop('rest', axis=1)

# df['word'] = df[0].str.extract(r'^([^(=\[\-,]+?)(?=\s*[\(=\[\-,]|$)', expand=False)

# pattern = r'^([^.]+)\.\s+(.*)$|^([^"]+)"\s*(.*)$|^([^/]+)/\s*(.*)$|^([^|]+)\|\s*(.*)$|^([^:]+):\s*(.*)$'
# df['word'] = df[0].str.extract(pattern, expand=False)[0]

df[['word', 'description']] = df[0].str.extract(r'^([^(=\[\-,]+?)(?:\s*[\(=\[\-,]\s*(.*))?$', expand=True)
df = df.drop(0, axis=1)

In [ ]:
step_0 = df
step_0.info()

In [ ]:
# Cleaning
#########################
df.drop_duplicates(inplace=True)
df = df[~df["word"].str.contains(r'-\d+-', regex=True, na=False)] # -number- deletion
df['description'] = df['description'].fillna('N/A')
#########################

In [ ]:
step_1 = df
step_1.info()

In [ ]:
df = df.dropna(subset=['word']) # NaN for word section

In [ ]:
df.info()

In [ ]:
# ##############################
# # Additional columns for better recognision
# new_columns = ['transcription','greek_analog']
# for col in new_columns:
#     df[col] = ''

# ########################

In [ ]:
df[df['description'] == 'N/A'].shape[0] # 2044 left, probably to drop, most of them are just page initials
df.drop(df[df['description'] == 'N/A'].index, inplace=True)

In [ ]:
df.info()

In [ ]:
# df[-30:-1]

In [ ]:
df.to_csv('clear_cypriot_dict.csv', index=False)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("ZiartisNikolas/NMT-cypriot-dialect-to-greek")
model = AutoModelForSeq2SeqLM.from_pretrained("ZiartisNikolas/NMT-cypriot-dialect-to-greek")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm import tqdm

# Загружаем модель
tokenizer = AutoTokenizer.from_pretrained("ZiartisNikolas/NMT-cypriot-dialect-to-greek")
model = AutoModelForSeq2SeqLM.from_pretrained("ZiartisNikolas/NMT-cypriot-dialect-to-greek")

In [ ]:
def translate_to_greek(text):
    if pd.isna(text) or text == '':
        return ''

    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512, num_beams=4, early_stopping=True)


    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation


In [ ]:
tqdm.pandas(desc="Translating")

df['greek_word'] = df['word'][:5].progress_apply(translate_to_greek)
df['greek_description'] = df['description'][:5].progress_apply(translate_to_greek)

print("Готово!")
print(df[['word', 'description', 'greek_translation']].head())

In [ ]:
all_greek = pd.read_csv('cyprus_greek_full_words.csv')
all_desc = pd.read_csv('combined_25000_desc.csv')

In [ ]:
all_greek.info()

In [ ]:
all_desc.info()

In [ ]:
combined = pd.concat([
    all_desc,
    all_greek['greek_word']
], axis=1, ignore_index=False)

combined = combined.drop(columns=['Unnamed: 0'], errors='ignore')

In [ ]:
combined.info()

In [ ]:
#################################
# Final Cleaning
combined.drop_duplicates(inplace=True)
# combined.dropna(inplace=True)
#################################

has_digits = combined['word'].str.contains(r'\d', na=False)